In [83]:
#load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Prepocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

#model training
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import VotingClassifier
import joblib

In [84]:
path = 'ObesityDataSet_raw.csv'
df_data = pd.read_csv(path)
df_data.head(100)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SCC,CALC,MTRANS,NObeyesdad
0,Female,21,1.62,64.0,yes,no,2,3,Sometimes,no,no,Public_Transportation,Normal_Weight
1,Female,21,1.52,56.0,yes,no,3,3,Sometimes,yes,Sometimes,Public_Transportation,Normal_Weight
2,Male,23,1.80,77.0,yes,no,2,3,Sometimes,no,Frequently,Public_Transportation,Normal_Weight
3,Male,27,1.80,87.0,no,no,3,3,Sometimes,no,Frequently,Walking,Overweight_Level_I
4,Male,22,1.78,89.8,no,no,2,1,Sometimes,no,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Male,26,1.70,70.0,yes,no,3,1,Frequently,no,Frequently,Public_Transportation,Normal_Weight
96,Female,23,1.66,60.0,yes,no,2,3,Sometimes,no,Sometimes,Public_Transportation,Normal_Weight
97,Female,21,1.52,42.0,no,no,3,1,Frequently,no,Sometimes,Public_Transportation,Insufficient_Weight
98,Female,21,1.52,42.0,no,no,3,1,Frequently,no,Sometimes,Public_Transportation,Insufficient_Weight


In [85]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   int64  
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   int64  
 7   NCP                             2111 non-null   int64  
 8   CAEC                            2111 non-null   object 
 9   SCC                             2111 non-null   object 
 10  CALC                            2111 non-null   object 
 11  MTRANS                          2111 non-null   object 
 12  NObeyesdad                      21

In [86]:
df_data.describe()

,Age,Height,Weight,FCVC,NCP
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.315964,1.701620,86.585978,2.423496,2.687826
std,6.357078,0.093368,26.191165,0.583905,0.809680
min,14.000000,1.450000,39.000000,1.000000,1.000000
25%,20.000000,1.630000,65.450000,2.000000,3.000000
50%,23.000000,1.700000,83.000000,2.000000,3.000000
75%,26.000000,1.770000,107.450000,3.000000,3.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000


In [87]:
# Checking for duplicated rows
duplicated_count = df_data.duplicated().sum()
print(f"Jumlah data duplikat: {duplicated_count}")

Jumlah data duplikat: 170


In [88]:
# drop duplicate
df_drop_duplicate = df_data.drop_duplicates()
duplicated_count = df_drop_duplicate.duplicated().sum()
print(f"Jumlah data duplikat: {duplicated_count}")

Jumlah data duplikat: 0


In [89]:
categorical_columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad']
le = LabelEncoder()
# Define the ordered labeling for each categorical column
label_orders = {
    'CAEC': {
        'no': 0,
        'Sometimes': 1,
        'Frequently': 2,
        'Always': 3
    },
    'MTRANS': {
        'Walking': 0,
        'Bike': 1,
        'Motorbike': 2,
        'Public_Transportation': 3,
        'Automobile': 4
    },
    # For other columns, you can specify their orders if needed
    # If no specific order is needed, they will be encoded alphabetically
    'Gender': {},'FHWO': {},'FAVC': {},'SMOKE': {},'SCC': {},
    'CALC': {'no': 0,
        'Sometimes': 1,
        'Frequently': 2,
        'Always': 3
    },            
    'NObeyesdad': { 'Insufficient_Weight' : 0,
                   'Normal_Weight' : 1, 
                   'Overweight_Level_I' : 2,  
                   'Overweight_Level_II' : 3,
                   'Obesity_Type_I' : 4, 
                   'Obesity_Type_II' :  5,  
                   'Obesity_Type_III' : 6
                   }
}

# Apply LabelEncoder with specified orders where applicable
for col in categorical_columns:
    if col in label_orders and label_orders[col]:
        # Create mapping for columns with specified order
        unique_values = df_drop_duplicate[col].unique()
        # Include all possible values in the classes_
        le.classes_ = list(label_orders[col].keys())
        df_drop_duplicate[col] = df_drop_duplicate[col].map(label_orders[col])
    else:
        # Standard LabelEncoder for columns without specified order
        df_drop_duplicate[col] = le.fit_transform(df_drop_duplicate[col])

C:\Users\User\AppData\Local\Temp\ipykernel_26428\3578793187.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop_duplicate[col] = le.fit_transform(df_drop_duplicate[col])
C:\Users\User\AppData\Local\Temp\ipykernel_26428\3578793187.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop_duplicate[col] = le.fit_transform(df_drop_duplicate[col])
C:\Users\User\AppData\Local\Temp\ipykernel_26428\3578793187.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [91]:
df_drop_duplicate.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SCC,CALC,MTRANS,NObeyesdad
0,0,21,1.62,64.0,1,0,2,3,1,0,0,3,1
1,0,21,1.52,56.0,1,0,3,3,1,1,1,3,1
2,1,23,1.80,77.0,1,0,2,3,1,0,2,3,1
3,1,27,1.80,87.0,0,0,3,3,1,0,2,0,2
4,1,22,1.78,89.8,0,0,2,1,1,0,1,3,3


In [92]:
numerical_columns = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CAEC', 'CALC', 'MTRANS']
# Inisialisasi StandardScaler
scaler = StandardScaler()

# Melakukan Standard Scaling pada kolom numerik yang relevan
df_drop_duplicate[numerical_columns] = scaler.fit_transform(df_drop_duplicate[numerical_columns])

print("\nDataFrame setelah StandardScaler:")
df_drop_duplicate.head(100)


DataFrame setelah StandardScaler:


C:\Users\User\AppData\Local\Temp\ipykernel_26428\3313473959.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop_duplicate[numerical_columns] = scaler.fit_transform(df_drop_duplicate[numerical_columns])


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SCC,CALC,MTRANS,NObeyesdad
0,0,-0.510579,-0.899998,-0.839675,1,0,-0.687305,0.371366,-0.325189,0,-1.373813,-0.185571,1
1,0,-0.510579,-1.963040,-1.144988,1,0,1.018652,0.371366,-0.325189,1,0.536338,-0.185571,1
2,1,-0.204326,1.013477,-0.343542,1,0,-0.687305,0.371366,-0.325189,0,2.446490,-0.185571,1
3,1,0.408179,1.013477,0.038099,0,0,1.018652,0.371366,-0.325189,0,2.446490,-4.490675,2
4,1,-0.357453,0.800869,0.144959,0,0,-0.687305,-2.114223,-0.325189,0,0.536338,-0.185571,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.255053,-0.049565,-0.610690,1,0,1.018652,-2.114223,1.807217,0,2.446490,-0.185571,1
96,0,-0.204326,-0.474782,-0.992331,1,0,-0.687305,0.371366,-0.325189,0,0.536338,-0.185571,1
97,0,-0.510579,-1.963040,-1.679285,0,0,1.018652,-2.114223,1.807217,0,0.536338,-0.185571,0
99,1,-0.204326,0.163044,-0.610690,0,0,-0.687305,0.371366,-0.325189,0,2.446490,-0.185571,1


In [93]:
selected_features = ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad']
df = df_drop_duplicate[selected_features]
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SCC,CALC,MTRANS,NObeyesdad
0,0,-0.510579,-0.899998,-0.839675,1,0,-0.687305,0.371366,-0.325189,0,-1.373813,-0.185571,1
1,0,-0.510579,-1.963040,-1.144988,1,0,1.018652,0.371366,-0.325189,1,0.536338,-0.185571,1
2,1,-0.204326,1.013477,-0.343542,1,0,-0.687305,0.371366,-0.325189,0,2.446490,-0.185571,1
3,1,0.408179,1.013477,0.038099,0,0,1.018652,0.371366,-0.325189,0,2.446490,-4.490675,2
4,1,-0.357453,0.800869,0.144959,0,0,-0.687305,-2.114223,-0.325189,0,0.536338,-0.185571,3


## Data Splitting

In [94]:
# Separate the dependent variable from the independent variables
y = df['NObeyesdad']
X = df.drop('NObeyesdad', axis=1)
# 2. Splitting 80:20 (80% train, 20% test)
X_train_80, X_test_20, y_train_80, y_test_20 = train_test_split(X, y, test_size=0.2, random_state=42)

In [95]:
# @title 80:20
# Print shapes of the resulting datasets
print("X_train:", X_train_80.shape)
print("X_test:", X_test_20.shape)
print("y_train:", y_train_80.shape)
print("y_test:", y_test_20.shape)

X_train: (1552, 12)
X_test: (389, 12)
y_train: (1552,)
y_test: (389,)


#80:20

## Best Parameter

### Decsision Tree

In [105]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_param_grid = {
    'max_depth': [10, 15, 20, 25],
    'min_samples_leaf': [1, 2, 5],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}
tree_grid_search = GridSearchCV(tree_clf, tree_param_grid, cv=10, n_jobs=-1, verbose=3)
tree_grid_search.fit(X_train_80, y_train_80)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 15, 20, 25],
                         'min_samples_leaf': [1, 2, 5],
                         'splitter': ['best', 'random']},
             verbose=3)

In [106]:
# Get best parameters and score
print("Best Parameters:", tree_grid_search.best_params_)
print("Best Cross-Validation Score: {:.4f}%".format(tree_grid_search.best_score_ * 100))

Best Parameters: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 1, 'splitter': 'best'}
Best Cross-Validation Score: 94.4586%


In [107]:
# Evaluate best Decision Tree on test set
best_tree_clf = tree_grid_search.best_estimator_
best_pred_dt = best_tree_clf.predict(X_test_20)
accuracy_dt_best = accuracy_score(y_test_20, best_pred_dt) * 100
print("Accuracy of Best Decision Tree: {:.6f}%".format(accuracy_dt_best))
print("Classification Report:\n", classification_report(y_test_20, best_pred_dt, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test_20, best_pred_dt))

Accuracy of Best Decision Tree: 96.401028%
Classification Report:
               precision    recall  f1-score   support

           0     0.9322    0.9483    0.9402        58
           1     0.9273    0.9107    0.9189        56
           2     0.9756    0.9091    0.9412        44
           3     0.9464    0.9815    0.9636        54
           4     0.9697    1.0000    0.9846        64
           5     1.0000    0.9841    0.9920        63
           6     1.0000    1.0000    1.0000        50

    accuracy                         0.9640       389
   macro avg     0.9645    0.9620    0.9629       389
weighted avg     0.9642    0.9640    0.9639       389

Confusion Matrix:
 [[55  3  0  0  0  0  0]
 [ 4 51  1  0  0  0  0]
 [ 0  1 40  3  0  0  0]
 [ 0  0  0 53  1  0  0]
 [ 0  0  0  0 64  0  0]
 [ 0  0  0  0  1 62  0]
 [ 0  0  0  0  0  0 50]]


### Random Forest

In [108]:
# Initialize Random Forest Classifier and GridSearchCV
rf_clf = RandomForestClassifier(random_state=42)
rf_param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'criterion': ['entropy', 'gini']
}
rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, verbose=3, cv=10, n_jobs=-1)
# Fit GridSearchCV
rf_grid_search.fit(X_train_80, y_train_80)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['entropy', 'gini'],
                         'n_estimators': [50, 100, 150, 200, 250]},
             verbose=3)

In [109]:
# Get best parameters and score
print("Best Parameters:", rf_grid_search.best_params_)
print("Best Cross-Validation Score: {:.4f}%".format(rf_grid_search.best_score_ * 100))

Best Parameters: {'criterion': 'gini', 'n_estimators': 250}
Best Cross-Validation Score: 94.2651%


In [110]:
# Evaluate best Random Forest on test set
best_rf_clf = rf_grid_search.best_estimator_
best_pred_rf = best_rf_clf.predict(X_test_20)
accuracy_rf_best = accuracy_score(y_test_20, best_pred_rf) * 100
print("Accuracy of Best Random Forest: {:.6f}%".format(accuracy_rf_best))
print("Classification Report:\n", classification_report(y_test_20, best_pred_rf, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test_20, best_pred_rf))

Accuracy of Best Random Forest: 95.629820%
Classification Report:
               precision    recall  f1-score   support

           0     0.9825    0.9655    0.9739        58
           1     0.8361    0.9107    0.8718        56
           2     0.9524    0.9091    0.9302        44
           3     0.9434    0.9259    0.9346        54
           4     0.9844    0.9844    0.9844        64
           5     1.0000    0.9841    0.9920        63
           6     1.0000    1.0000    1.0000        50

    accuracy                         0.9563       389
   macro avg     0.9570    0.9543    0.9553       389
weighted avg     0.9580    0.9563    0.9568       389

Confusion Matrix:
 [[56  2  0  0  0  0  0]
 [ 1 51  2  2  0  0  0]
 [ 0  4 40  0  0  0  0]
 [ 0  4  0 50  0  0  0]
 [ 0  0  0  1 63  0  0]
 [ 0  0  0  0  1 62  0]
 [ 0  0  0  0  0  0 50]]


### Logistic Regession

In [111]:
# Initialize Logistic Regression with increased max_iter
lr_clf = LogisticRegression(max_iter=1000)

lr_param_grid = [
    # For L1 penalty (only liblinear and saga support L1)
    {
        'penalty': ['l1'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['liblinear', 'saga']
    },
    # For L2 penalty (all solvers support L2)
    {
        'penalty': ['l2'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['liblinear', 'saga', 'lbfgs']
    },
    # For no penalty (None)
    {
        'penalty': [None],
        'solver': ['lbfgs', 'saga']
    }
]
# Initialize GridSearchCV
lr_grid_search = GridSearchCV(lr_clf, lr_param_grid, verbose=3, cv=10, n_jobs=-1)
lr_grid_search.fit(X_train_80, y_train_80)

Fitting 10 folds for each of 22 candidates, totalling 220 fits


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 10], 'penalty': ['l1'],
                          'solver': ['liblinear', 'saga']},
                         {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'],
                          'solver': ['liblinear', 'saga', 'lbfgs']},
                         {'penalty': [None], 'solver': ['lbfgs', 'saga']}],
             verbose=3)

In [112]:
# Get best parameters and score
print("Best Parameters:", lr_grid_search.best_params_)
print("Best Cross-Validation Score: {:.2f}%".format(lr_grid_search.best_score_ * 100))

Best Parameters: {'C': 10, 'penalty': 'l1', 'solver': 'saga'}
Best Cross-Validation Score: 96.13%


In [113]:
# Evaluate best Logistic Regression
best_lr_clf = lr_grid_search.best_estimator_
best_pred_lr = best_lr_clf.predict(X_test_20)
accuracy_lr_best = accuracy_score(y_test_20, best_pred_lr) * 100
print("Accuracy of Best Logistic Regression: {:.6f}%".format(accuracy_lr_best))
print("Classification Report:\n", classification_report(y_test_20, best_pred_lr, digits = 4))
print("Confusion Matrix:\n", confusion_matrix(y_test_20, best_pred_lr))

Accuracy of Best Logistic Regression: 96.401028%
Classification Report:
               precision    recall  f1-score   support

           0     0.9667    1.0000    0.9831        58
           1     0.9818    0.9643    0.9730        56
           2     0.9286    0.8864    0.9070        44
           3     0.9245    0.9074    0.9159        54
           4     0.9688    0.9688    0.9688        64
           5     0.9692    1.0000    0.9844        63
           6     1.0000    1.0000    1.0000        50

    accuracy                         0.9640       389
   macro avg     0.9628    0.9610    0.9617       389
weighted avg     0.9637    0.9640    0.9637       389

Confusion Matrix:
 [[58  0  0  0  0  0  0]
 [ 2 54  0  0  0  0  0]
 [ 0  1 39  4  0  0  0]
 [ 0  0  3 49  2  0  0]
 [ 0  0  0  0 62  2  0]
 [ 0  0  0  0  0 63  0]
 [ 0  0  0  0  0  0 50]]


### Hasil Best Param

## Ensemble Voting

### Soft Voting

In [114]:
# Create VotingClassifier with soft voting
voting_clf_soft = VotingClassifier(
    estimators=[
        ('tree', best_tree_clf),
        ('rf', best_rf_clf),
        ('lr', best_lr_clf)
    ],
    voting='soft'
)

# Fit the VotingClassifier on the training data
voting_clf_soft.fit(X_train_80, y_train_80)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


VotingClassifier(estimators=[('tree',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=15,
                                                     random_state=42)),
                             ('rf',
                              RandomForestClassifier(n_estimators=250,
                                                     random_state=42)),
                             ('lr',
                              LogisticRegression(C=10, max_iter=1000,
                                                 penalty='l1',
                                                 solver='saga'))],
                 voting='soft')

In [115]:
voting_soft = voting_clf_soft.predict(X_test_20)
acc_soft_voting = accuracy_score(y_test_20, voting_soft) * 100
print("Accuracy of Soft Voting 3_1: {:.6f}%".format(acc_soft_voting))
print("Classification Report:\n", classification_report(y_test_20, voting_soft, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test_20, voting_soft))

Accuracy of Soft Voting 3_1: 97.429306%
Classification Report:
               precision    recall  f1-score   support

           0     0.9500    0.9828    0.9661        58
           1     0.9630    0.9286    0.9455        56
           2     0.9762    0.9318    0.9535        44
           3     0.9636    0.9815    0.9725        54
           4     0.9697    1.0000    0.9846        64
           5     1.0000    0.9841    0.9920        63
           6     1.0000    1.0000    1.0000        50

    accuracy                         0.9743       389
   macro avg     0.9746    0.9727    0.9734       389
weighted avg     0.9745    0.9743    0.9742       389

Confusion Matrix:
 [[57  1  0  0  0  0  0]
 [ 3 52  1  0  0  0  0]
 [ 0  1 41  2  0  0  0]
 [ 0  0  0 53  1  0  0]
 [ 0  0  0  0 64  0  0]
 [ 0  0  0  0  1 62  0]
 [ 0  0  0  0  0  0 50]]


In [116]:
import pickle

In [117]:
filename = 'obesity_prediction.sav'
pickle.dump(voting_clf_soft, open(filename,'wb'))

In [118]:
joblib.dump(voting_clf_soft, 'voting_clf_soft.joblib')
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']